<a href="https://www.kaggle.com/code/kaizor11/cleaning-tweet-sentiments-and-stocks-prediction?scriptVersionId=211309115" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import warnings
warnings.filterwarnings("ignore")

import time
import pandas as pd
import numpy as np

df = pd.read_csv("/kaggle/input/tweet-sentiment-s-impact-on-stock-returns/full_dataset-release.csv")
df2 = pd.read_csv("/kaggle/input/tweet-sentiment-s-impact-on-stock-returns/reduced_dataset-release.csv")

In [3]:
df

,Unnamed: 0,TWEET,STOCK,DATE,LAST_PRICE,1_DAY_RETURN,2_DAY_RETURN,3_DAY_RETURN,7_DAY_RETURN,PX_VOLUME,VOLATILITY_10D,VOLATILITY_30D,LSTM_POLARITY,TEXTBLOB_POLARITY
0,0,RT @robertoglezcano: @amazon #Patents Show Fl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Amazon,31/01/2017,823.48,0.008379,0.014924,0.014924,-0.001263,3.137196e+06,13.447,16.992,1.000,0.00000,NaN
2,1,@FAME95FM1 Jamaicans make money with @Payoneer...,PayPal,31/01/2017,39.780000,0.002011,0.012318,0.012318,5.480141e-02,9100057.000,18.769,16.099,-1.00000,0.0
3,2,@CBSi Jamaicans make money with @Payoneer @Pay...,PayPal,31/01/2017,39.780000,0.002011,0.012318,0.012318,5.480141e-02,9100057.000,18.769,16.099,1.00000,0.0
4,3,@Hitz92fm Jamaicans make money with @Payoneer ...,PayPal,31/01/2017,39.780000,0.002011,0.012318,0.012318,5.480141e-02,9100057.000,18.769,16.099,-1.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395445,NaN,PayPal,30/09/2018,87.84,0.000000,0.000000,0.013092,0.031648,7.920916e+06,15.754,23.131,-1.000,0.20000,NaN
1395446,862229.0,RT @RifatJawaid: Deceased Apple executive's br...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1395447,NaN,Apple,30/09/2018,225.74,0.000000,0.000000,-0.003500,-0.035793,2.292936e+07,15.841,20.065,1.000,0.00000,NaN
1395448,862230.0,Flat 6000 INR cashback on HDFC debit and credi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# EDA

In [ ]:
df_temp = df.copy()
df_temp = df_temp.dropna()
df_temp = df_temp.drop_duplicates()

count = df_temp['STOCK'].value_counts()
perc = count/count.sum()
dstr = pd.DataFrame({"Count": count, "Percentage": perc}).reset_index()
dstr

In [ ]:
import matplotlib.pyplot as plt
plt.pie(dstr['Count'], labels = dstr['STOCK'])
plt.show()

In [ ]:
# Top 5 stocks (~50% share of tweets)
TOP_STOCKS = dstr['STOCK'].head().to_list()

# Middle 25% 
MID_STOCKS = dstr['STOCK'].iloc[5:].head().to_list()

# Bottom 25% (Below Starbucks)
BOT_STOCKS = dstr['STOCK'].iloc[10:].to_list()

# Cleaning

In [ ]:
# COL = ['Ford', 'Facebook', 'Google']
df_top = df[df['STOCK'].isin(TOP_STOCKS) | df['TWEET'].isin(TOP_STOCKS)]
df_top = df_top.reset_index(drop=True)
l1 = len(df_top)
l2 = len(df_top[df_top['Unnamed: 0'].isnull()])
df_top[df_top['Unnamed: 0'].isnull()]
print(f"Expected cleaned df size: {l1 - l2}")

df_mid = df[df['STOCK'].isin(MID_STOCKS) | df['TWEET'].isin(MID_STOCKS)]
df_mid = df_mid.reset_index(drop=True)
l1 = len(df_mid)
l2 = len(df_mid[df_mid['Unnamed: 0'].isnull()])
df_mid[df_mid['Unnamed: 0'].isnull()]
print(f"Expected cleaned df size: {l1 - l2}")

df_bot = df[df['STOCK'].isin(BOT_STOCKS) | df['TWEET'].isin(BOT_STOCKS)]
df_bot = df_bot.reset_index(drop=True)
l1 = len(df_bot)
l2 = len(df_bot[df_bot['Unnamed: 0'].isnull()])
df_bot[df_bot['Unnamed: 0'].isnull()]
print(f"Expected cleaned df size: {l1 - l2}")

In [ ]:
# Function to fix misalignment
def fix_misalignment(df):
    # Copy the original DataFrame to avoid altering it directly
    df = df.copy()

    # Create a mask where the first column is NaN
    mask = df.iloc[:, 0].isna()

    # Iterate over rows where the mask is True
    for idx in df[mask].index.tolist():  # Use a list of indices to avoid dynamic changes
        # Get the previous row index
        previous_idx = idx - 1
        
        # Ensure the previous index is valid and within bounds
        if previous_idx in df.index:  
            for col in range(1, len(df.columns)):
                # Check if the current value is non-NaN
                if not pd.isna(df.iloc[idx, col]):
                    # Ensure the column is within bounds of the previous row
                    if col+1 < len(df.columns):
                        df.iloc[previous_idx, col+1] = df.iloc[idx, col]

    # Drop rows where the first column is NaN
    df = df[~mask]

    return df

In [ ]:
# 22 MINUTE RUNTIME

# BOT
start_time = time.time()

df_bot_cleaned = fix_misalignment(df_bot)

end_time = time.time()
runtime = end_time - start_time
print(f'RUNTIME: {runtime:.2f} seconds')

# MID
start_time = time.time()

df_mid_cleaned = fix_misalignment(df_mid)

end_time = time.time()
runtime = end_time - start_time
print(f'RUNTIME: {runtime:.2f} seconds')

# TOP
start_time = time.time()

df_top_cleaned = fix_misalignment(df_top)

end_time = time.time()
runtime = end_time - start_time
print(f'RUNTIME: {runtime:.2f} seconds')

In [ ]:
df_top_cleaned = df_top_cleaned.reset_index(drop=True)
df_top_cleaned = df_top_cleaned.drop('Unnamed: 0', axis=1)
df_top_cleaned

In [ ]:
df_mid_cleaned = df_mid_cleaned.reset_index(drop=True)
df_mid_cleaned = df_mid_cleaned.drop('Unnamed: 0', axis=1)
df_mid_cleaned

In [ ]:
df_bot_cleaned = df_bot_cleaned.reset_index(drop=True)
df_bot_cleaned = df_bot_cleaned.drop('Unnamed: 0', axis=1)
df_bot_cleaned

In [ ]:
df_top_cleaned.to_csv('/kaggle/working/df_top_cleaned.csv', index=False)
df_mid_cleaned.to_csv('/kaggle/working/df_mid_cleaned.csv', index=False)
df_bot_cleaned.to_csv('/kaggle/working/df_bot_cleaned.csv', index=False)

#